## Использование Universal Sentence Encoder для получения эмбеддингов названий компаний

Загрузим тренировочный датасет

In [1]:
import pandas as pd
import numpy as np
train_ds = pd.read_csv('https://raw.githubusercontent.com/vladseve7n/ITMO.DUBL/main/dataset/train.csv').drop('pair_id', axis=1)
train_ds

,name_1,name_2,is_duplicate
0,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,Powermax Rubber Factory,Co. One,0
4,Tress A/S,Longyou Industries Park Zhejiang,0
...,...,...,...
497814,BIT-MAT PRODUCTS,The Goodyear Tire and Rubber Company,0
497815,"Bnd Trading Co., Ltd.",Zhong Shan Yue Liang Economy& Trade Imp. & Exp...,0
497816,"Xeikon Industrial Co., Ltd. Of Dongguan City","Yi Cheng Trading Co., Ltd. Of Dongguan City",0
497817,"Shanghai Kechuan Trading Co., Ltd.",Shanghai M&G Stationery Inc.,0


Загрузим предварительно запарсенные кластеры

In [2]:
import requests
clusters = requests.get('https://raw.githubusercontent.com/vladseve7n/ITMO.DUBL/main/dataset/clusters.json').json()['clusters']

Скачаем модель USE с tensorflow_hub

In [3]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

2022-10-24 14:13:51.559828: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-24 14:13:51.934097: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-24 14:13:52.705056: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/usersp/miniconda3/lib/
2022-10-24 14:13:52.705168: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: 

Получим эмбеддинг для каждого уникального названия в датасете

In [4]:
unique_names = list(np.unique(np.append(train_ds.name_1.values, train_ds.name_2.values)))
print(f'Всего уникальных имен {len(unique_names)}')

Всего уникальных имен 18022


In [5]:
%%time
embeddings_dict = {name: embed([name])[0].numpy() for name in unique_names}

2022-10-24 14:13:58.738262: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-10-24 14:13:58.738284: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-10-24 14:13:58.738299: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update

CPU times: user 1min 28s, sys: 12.2 s, total: 1min 40s
Wall time: 45.6 s


Посмотрим на расстояние между парами в датасете

In [6]:
train_ds['l1'] = (abs(train_ds.name_2.apply(lambda x: embeddings_dict[x]) - train_ds.name_1.apply(lambda x: embeddings_dict[x]))).apply(sum)
train_ds['l2'] = ((train_ds.name_2.apply(lambda x: embeddings_dict[x]) - train_ds.name_1.apply(lambda x: embeddings_dict[x]))**2).apply(sum)**(1/2)

In [23]:
train_ds[train_ds['is_duplicate'] == 1]

,name_1,name_2,is_duplicate,l1,l2
161,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Technical Serv...,1,13.198858,0.774979
603,Pirelli Neumaticos S.A.I.C.,"Pirelli Tyre Co., Ltd.",1,15.278474,0.861093
835,Brenntag Australia (Pty) Ltd.,Brenntag Group,1,23.086198,1.246404
1328,"PAUL BAUDER GMBH & CO KG, BOCHUM PLANT",Paul Bauder ag,1,16.303659,0.937461
1562,TOTAL CESKA REPUBLIKA s.r.o.,TOTAL FRANCE (ARNAY LE DUC),1,21.662789,1.192262
...,...,...,...,...,...
496574,"Bridgestone （Huizhou）Synthetic Rubber Co., Ltd.","Bridgestone India Pvt., Ltd.",1,15.799219,0.911611
496760,Arlanxeo International Group,Arlanxeo Corp.,1,19.748249,1.105423
497015,Brenntag Peru S.A.C.,Brenntag Chile Comercial E Industrial Ltda,1,20.787460,1.173913
497083,Dow Chemical International Private Ltd.,Dow Chemical Pacific,1,14.903662,0.847723


In [8]:
train_ds[train_ds['is_duplicate'] == 1].l1.mean(), train_ds[train_ds['is_duplicate'] == 1].l2.mean()

(17.36295437039792, 0.9751832977954248)

In [9]:
train_ds[train_ds['is_duplicate'] == 0].l1.mean(), train_ds[train_ds['is_duplicate'] == 0].l2.mean()

(21.619643283359217, 1.1929991510725755)

Создадим датасет со средним вектором каждого кластера и попробуем найти соответствие по минимальному расстоянию

In [10]:
import numpy as np
from tqdm import tqdm

In [11]:
cluster_embedding = {}
for num, cluster in enumerate(tqdm(clusters)):
    cluster_embedding[num] = {}
    cluster_embedding[num]['cluster'] = cluster
    embed_cluster = np.mean(np.array([embeddings_dict[x] for x in cluster]), axis=0)
    cluster_embedding[num]['embedding'] = embed_cluster

100%|██████████████████████████████████████████████████████████████| 450/450 [00:00<00:00, 67127.96it/s]


In [25]:
def find_closest_cluster(cluster_embedding: dict, name_of_company: str, embed, distance: str = 'l1'):
    name_embedding = embed([name_of_company])[0].numpy()
    min_distance = 10e12
    result_cluster = None
    for key in cluster_embedding:
        cluster = cluster_embedding[key]['cluster']
        cluster_embed = cluster_embedding[key]['embedding']
        if distance == 'l1':
            distance = np.abs(cluster_embed - name_embedding).sum()
        else:
            distance = (((cluster_embed - name_embedding)**2).sum())**(1/2)
        if distance < min_distance:
            min_distance = distance
            result_cluster = cluster
    return result_cluster, min_distance

In [27]:
find_closest_cluster(cluster_embedding, 'Paul Bauder ag	', embed, distance='l2')

(['PAUL BAUDER GMBH & CO KG',
  'PAUL BAUDER GMBH & CO KG, BOCHUM PLANT',
  'PAUL BAUDER GMBH & CO KG, LANDSBERG PLANT',
  'PAUL BAUDER GMBH & CO KG, STUTTGART PLANT',
  'Paul Bauder ag',
  'PAUL BAUDER GMBH & CO.KG, ACHIM PLANT',
  'Paul Bauder GmbH & Co. KG'],
 0.8561134869980347)

In [28]:
find_closest_cluster(cluster_embedding, 'Bridgestone International Group', embed, distance='l2')

(['Bridgestone India Pvt., Ltd.',
  'Bridgestone Canada Inc.',
  'Bridgestone Golf (Shenzhen) Co., Ltd.',
  'Bridgestone India Automotive',
  'Bridgestone(Shenyang) Tire Co., Ltd.',
  'Bridgestone (Tianjin) Tire Co., Ltd.',
  'Warren Plant Bato Bridgestone',
  'Bridgestone Firestone Venezolana',
  'Bridgestone De Costa Rica Sociedad Anoni',
  'Bridgestone International Group',
  'Bridgestone (Huizhou) Tire Co., Ltd.',
  'Pt Bridgestone Tire Indonesia',
  'Bridgestone (China) Research And Development Co., Ltd.',
  'Bridgestone Firestone De Mexico Sa De Cv',
  'Bridgestone Tire',
  'Thai Bridgestone Co., Ltd.',
  'Bridgestone De Mexico S.A. De C.V.',
  'Bridgestone （Huizhou）Synthetic Rubber Co., Ltd.',
  'Bridgestone India',
  'Bridgestone Do Brasil Industria E Comercio Ltda',
  'Bridgestone Americas Tire',
  'Bridgestone Firestone Do Brasil',
  'Bridgestone Do Brasil Industria & Comercio Ltda',
  'Bridgestone Firestone De Argentina Saic',
  'Bridgestone Ncr',
  'Bridgestone Hosepower Ll